# Backup

In [7]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import dateutil.parser
import time
import random


In [8]:
# pd.read_csv('g_movies_df.csv')  

In [9]:
# read pickle file as dataframe
g_movies_df = pd.read_pickle('g_movies_df.pkl')
g_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, Toy Story 4 to WALL·E
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   link_stub                  9 non-null      object        
 1   title                      9 non-null      object        
 2   rank_g_movies              9 non-null      object        
 3   lifetime_gross             9 non-null      object        
 4   rank_overall               9 non-null      object        
 5   year                       9 non-null      object        
 6   domestic_total_gross       9 non-null      int64         
 7   international_total_gross  9 non-null      int64         
 8   worldwide_total_gross      9 non-null      int64         
 9   domestic_opening           9 non-null      int64         
 10  budget                     8 non-null      object        
 11  release_date               9 non-null      datetime64[ns]
 12  ru

In [10]:
g_movies_df

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year,domestic_total_gross,international_total_gross,worldwide_total_gross,domestic_opening,budget,release_date,runtime,distributor,genres,rating
Toy Story 4,/title/tt1979376/?ref_=bo_cso_table_1,Toy Story 4,1,"$434,038,008",25,2019,434038008,639356585,1073394593,120908065,"$200,000,000",2019-06-20,100,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,G
The Lion King,/title/tt0110357/?ref_=bo_cso_table_2,The Lion King,2,"$422,783,777",29,1994,422783777,640828028,1063611805,1586753,"$45,000,000",1994-06-15,88,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,G
Toy Story 3,/title/tt0435761/?ref_=bo_cso_table_3,Toy Story 3,3,"$415,004,880",31,2010,415004880,651965931,1066970811,110307189,"$200,000,000",2010-06-16,103,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,None
Finding Nemo,/title/tt0266543/?ref_=bo_cso_table_4,Finding Nemo,4,"$380,843,261",44,2003,380843261,559509384,940352645,70251710,"$94,000,000",2003-05-30,100,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,None
"Monsters, Inc.",/title/tt0198781/?ref_=bo_cso_table_5,"Monsters, Inc.",5,"$290,642,256",101,2001,290642256,289065482,579707738,62577067,"$115,000,000",2001-11-02,92,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,G
Monsters University,/title/tt1453405/?ref_=bo_cso_table_6,Monsters University,6,"$268,492,764",112,2013,268492764,475066881,743559645,82429469,None,2013-06-19,104,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,G
Toy Story 2,/title/tt0120363/?ref_=bo_cso_table_7,Toy Story 2,7,"$245,852,179",135,1999,245852179,251523202,497375381,300163,"$90,000,000",1999-11-19,92,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,None
Cars,/title/tt0317219/?ref_=bo_cso_table_8,Cars,8,"$244,082,982",137,2006,244082982,217908885,461991867,60119509,"$120,000,000",2006-06-08,117,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,None
WALL·E,/title/tt0910970/?ref_=bo_cso_table_9,WALL·E,9,"$223,808,164",167,2008,223808164,297503726,521311890,63087526,"$180,000,000",2008-06-26,98,Walt Disney Studios Motion PicturesSee full co...,Adventure\n \n Animation\n \n ...,None


In [11]:
# function to create movies dict to collect values from get_movie_value function from above

def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page, "lxml")
    else:
      print(f"Oops! Received status code {status}")
    
    
    headers = ['movie_title', 
               'domestic_total_gross',
               'international_total_gross',
               'worldwide_total_gross',
               'domestic_opening',
               'budget',
               'release_date',
               'runtime',
               'distributor',
               'genres',
               'rating'
              ]

    #get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    print(title)
    ## GROSS VALUES from performance summary table on left side of each movie page
    try: #get domestic gross     
        domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[0]
                                        .text)
        print('dtg', domestic_total_gross )
    except:
        return None 

#     try: # convert domestic gross to int       
#         domestic_total_gross = money_to_int(raw_domestic_total_gross)
#         print('DTG', domestic_total_gross )
#     except:
#         return None 
    
    try: # get international gross
        international_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[1]
                                        .text)
        print('itg', international_total_gross)
    except:
        return None 
   
#     try:  #convert international gross to int    
#         international_total_gross = money_to_int(raw_international_total_gross)
#     except:
#         return None 
    
    try: # get worldwide gross
        worldwide_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[2]
                                        .text)
        print('wwg',worldwide_total_gross )
    except:
        return None 
    
#     try: # convert worldwide gross to int
#         worldwide_total_gross = money_to_int(raw_worldwide_total_gross)
#         print('WTG', worldwide_total_gross)
#     except:
#         return None    
    
    ## SUMMARY DETAILS from table in the center of each movie page
    try: # get domestic opening
        domestic_opening = get_movie_value(soup,'Domestic Opening')
    except:
        return None 

#     try: # convert domestic opening to int
#         domestic_opening = money_to_int(raw_domestic_opening)
#         print('DO', domestic_opening)
#     except:
#         return None 
    
    try: # get budget
        budget = get_movie_value(soup,'Budget')
        # convert budget to int
        # budget = money_to_int(raw_budget)
        print('budget', budget)
    except:
        return None 

    try: #get release date
        raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
        release_date = to_date(raw_release_date)
        print('relase date', release_date)
    except:
        return None 

    try: #get runtime
        raw_runtime = get_movie_value(soup,'Running')
    except:
        return None 
    
    try: #convert runtime to minutes
        runtime = runtime_to_minutes(raw_runtime)
        print('runtime', runtime)
    except:
        return None     

    try: # get distributor
        raw_distributor = get_movie_value(soup,'Distributor')
        distributor = raw_distributor.replace('See full company information', '').replace('\n\n', '')
        print('distributor', distributor)
    except:
        return None 

    try: # get genres
        genres = get_movie_value(soup,'Genres').replace('\n    \n        ', ', ')
        print('genres', genres)
    except:
        return None 

    try: #get rating
        rating = get_movie_value(soup,'MPAA')
        print('rating', rating)
    except:
        return None
        print('no rating')
    
    #create movie dictionary
    movie_dict = dict(zip(headers, [title,
                                    domestic_total_gross,
                                    international_total_gross,
                                    worldwide_total_gross,
                                    domestic_opening,
                                    budget,
                                    release_date,
                                    runtime,
                                    distributor,
                                    genres,
                                    rating]))

    return movie_dict

In [ ]:
# 2022-03-12 @ 3:40 
# added back in data cleaning statements in one try/except for each movie value

# function to create movies dict to collect values from get_movie_value function from above

def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page, "lxml")
    else:
      print(f"Oops! Received status code {status}")
    
    
    headers = ['movie_title', 
               'domestic_total_gross',
               'international_total_gross',
               'worldwide_total_gross',
               'domestic_opening',
               'budget',
               'release_date',
               'runtime',
               'distributor',
               'genres',
               'rating'
              ]

    #get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    print(title) # make sure scraper is working

    '''
    Get GROSS VALUES from performance summary table on left side of each movie page
    '''
 
    # domestic $$$
    try: #get raw domestic gross     
        raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[0].text)
        # convert raw domestic gross to int       
        domestic_total_gross = money_to_int(raw_domestic_total_gross)
    except:
    
    # international $$$
    try: # get raw international gross
        raw_international_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[1].text)
        #convert raw international gross to int    
        international_total_gross = money_to_int(raw_international_total_gross)
    except:
        return None
    
    # worldwide $$$
    try: # get raw worldwide gross
        raw_worldwide_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[2]
                                        .text)
        # convert raw worldwide gross to int
        worldwide_total_gross = money_to_int(raw_worldwide_total_gross)
    except:
        return None #BREAKINGLOOP 

    '''
    Get MOVIE DETAILS from summary details table in center of each movie page
    '''
    
    # domestic opening $$$ 
    try: # get domestic opening
        raw_domestic_opening = get_movie_value(soup,'Domestic Opening')
        # convert domestic opening to int
        domestic_opening = money_to_int(raw_domestic_opening)
    except:
        return None
    
    # budget $$$
    try: # get budget
        raw_budget = get_movie_value(soup,'Budget')
        # convert budget to int
        budget = money_to_int(raw_budget)
    except:
        return None
    
    try: #get release date
        raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
        release_date = to_date(raw_release_date)
    except:
        return None 

    try: #get raw runtime and convert to minutes
        raw_runtime = get_movie_value(soup,'Running')
        runtime = runtime_to_minutes(raw_runtime)
    except:
        return None 
    
    try: # get distributor and clean up 
        raw_distributor = get_movie_value(soup,'Distributor')
        distributor = raw_distributor.replace('See full company information', '').replace('\n\n', '')
    except:
        return None 

    try: # get genres and separate by commas
        genres = get_movie_value(soup,'Genres').replace('\n    \n        ', ', ')
    except:
        return None 

    try: #get rating
        rating = get_movie_value(soup,'MPAA')
    except:
        return None
    
    #create movie dictionary
    movie_dict = dict(zip(headers, [title,
                                    domestic_total_gross,
                                    international_total_gross,
                                    worldwide_total_gross,
                                    domestic_opening,
                                    budget,
                                    release_date,
                                    runtime,
                                    distributor,
                                    genres,
                                    rating]))

    return movie_dict